# COLLECT FUNDAMENTAL DATA FROM ALPHA VANTAGE

In [1]:
import pandas as pd
import requests

In [68]:
api_key = 'GZWRUSYXT18ZTR6C'
features_cols = ['fiscalDateEnding','totalRevenue','costOfRevenue','sellingGeneralAndAdministrative','researchAndDevelopment','depreciation','interestExpense','totalCurrentLiabilities','incomeTaxExpense','netIncome','commonStockSharesOutstanding','cashAndCashEquivalentsAtCarryingValue','cashAndShortTermInvestments','operatingCashflow','totalLiabilities','inventory','currentNetReceivables','propertyPlantEquipment','capitalExpenditures','longTermInvestments','totalShareholderEquity','longTermDebt','retainedEarnings','dividendPayoutCommonStock','paymentsForRepurchaseOfCommonStock','treasuryStock','currentLongTermDebt']
price_cols = ['open','high','low','close','volume']
DOW_30_TICKER = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']

## Collect for SP500 from Alpha Vantage

In [3]:
def collect_fundamental(ticker):
    # Download income statement
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data_is = r.json()

    # Download balance sheet
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data_bs = r.json()

    # Download cash flow
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={api_key}'
    r = requests.get(url)
    data_cf = r.json()

    df_is = pd.json_normalize(data_is['annualReports'])
    df_bs = pd.json_normalize(data_bs['annualReports'])
    df_cf = pd.json_normalize(data_cf['annualReports'])

    merged_df = df_is.merge(df_bs).merge(df_cf)
    merged_df['tic'] = ticker
    merged_df = merged_df[['tic']+features_cols]
    merged_df['fiscalDateEnding'] = pd.to_datetime(merged_df.fiscalDateEnding,format='mixed')

    return merged_df

Check reach download limitation

In [4]:
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=GS&apikey=GUKB8WJD0YFELJD3'
r = requests.get(url)
data_is = r.json()
data_is

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [ ]:
fund_data = pd.DataFrame()
tickets = DOW_30_TICKER[7:15]
for ticket in tickets:
    df_fund = collect_fundamental(ticket)
    fund_data = pd.concat([fund_data, df_fund], ignore_index=True)

In [ ]:
fund_data.to_csv('dow30.csv')

## Collect news for Dow30 from Alpha Vantage

In [59]:
time_from = '20210101T0130'
DOW_30_TICKER = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']

In [30]:
def collect_news(ticker):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&time_from={time_from}&tickers={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    feed = pd.json_normalize(data['feed'])
    feed['tic'] = ticker
    return feed

In [60]:
news_data = pd.DataFrame()

tickers = DOW_30_TICKER
for ticker in tickers:
    df_news = collect_news(ticker)
    news_data = pd.concat([news_data, df_news], ignore_index=True)

In [65]:
news_data.to_csv('dow30_news_data.csv')

## Collect Commondity prices from Alpha Vantage

In [74]:
COMMONDITY_CODE = ['WTI','BRENT','NATURAL_GAS','COPPER','ALUMINUM','WHEAT','CORN','COTTON','SUGAR','COFFEE','ALL_COMMODITIES']

In [78]:
def collect_commondity(ticker):
    url = f'https://www.alphavantage.co/query?function={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    commondity = pd.json_normalize(data['data'])
    commondity['symbol'] = ticker
    return commondity

In [79]:
commondity_data = pd.DataFrame()

for ticker in COMMONDITY_CODE:
    df_commondity = collect_commondity(ticker)
    commondity_data = pd.concat([commondity_data, df_commondity], ignore_index=True)

In [81]:
commondity_data.to_csv('commondity_data.csv')

## Collect Commondity prices from Alpha Vantage

In [82]:
ECONOMIC_CODE = ['REAL_GDP','REAL_GDP_PER_CAPITA','TREASURY_YIELD','FEDERAL_FUNDS_RATE','CPI','INFLATION','RETAIL_SALES','DURABLES','UNEMPLOYMENT','NONFARM_PAYROLL']

In [83]:
def collect_economic(ticker):
    url = f'https://www.alphavantage.co/query?function={ticker}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    economic = pd.json_normalize(data['data'])
    economic['symbol'] = ticker
    return economic

In [84]:
economic_data = pd.DataFrame()

for ticker in COMMONDITY_CODE:
    df_economic = collect_economic(ticker)
    economic_data = pd.concat([economic_data, df_economic], ignore_index=True)

In [85]:
economic_data.to_csv('economic_data.csv')

## Collect Technical Indicators from Alpha Vantage

## Vietnam stock market from VnStock

In [4]:
df_fund_vn = pd.read_csv('FUNDAMENTAL_combined_data_SECOND_RAW.csv')
df_fund_vn